In [1]:
import pandas as pd
import itertools
import numpy as np
import openpyxl
# from scenario_generation import generate_fuel_prices


### SETS

In [2]:
fuel_price_csv = 'fuel_price2_27s.csv'
route_case = 'stavanger'
mappenavn = 'generated_datafiles/'
startfil = ''
antall_ruter = 858
clusters = 11
scenarios = 27
# Velg antall av de ulike settene her
#set_sheets = {'Power systems': 11, 'Ages': 6, 'Routes': 3122, 'Time periods':6, 'Time periods 1': 2, 'Time periods 2':4, 'Fuel types': 10, 'Installations': 18, 'Scenarios': 100}

t_values = range(0, 6)  # time period from 1 to 6
t1_values = range(0, 2)
t2_values = range(2, 6)
tp2_values = range(2,7)
f_values = range(1, 7)  # Assuming you want values from 1 to 7 for 'f'
r_values = range(1, antall_ruter + 1)  # Assuming you want values from 1 to 3122 for 'r'
n_values = range(1, clusters + 1) # 18 installations
s_values = range(1,14) # 11 systems
w_values = range(1, scenarios + 1) # 100 scenarios
a_values = range(0, 6)

for t in f_values:
    print(t)


1
2
3
4
5
6


In [3]:
# Skriver alle sets med verdier definert over til excel

sets_excel_output_path = '../run_model/SetData-sheets.xlsx'

set_sheets = {
    'Power systems': s_values,
    'Ages': a_values,
    'Routes': r_values,
    'Time periods': t_values,
    'Time periods 1': t1_values,
    'Time periods 2': t2_values,
    'TP2 scrap': tp2_values,
    'Fuel types': f_values,
    'Installations': n_values,
    'Scenarios': w_values
}

# Create data frames for each set and write to Excel
with pd.ExcelWriter(sets_excel_output_path, engine='openpyxl') as writer:
    for sheet, values in set_sheets.items():
        # Create a DataFrame for the current set
        df = pd.DataFrame(itertools.product(values), columns=[sheet])

        # Write the DataFrame to the Excel file with the current sheet name
        df.to_excel(writer, sheet_name=sheet, index=False)

#### Generere kombinasjoner av indekser til parameterne

In [4]:

combinations_f_r = itertools.product(f_values, r_values)
combinations_ems = itertools.product(s_values, r_values)
combinations_r_s = itertools.product(r_values, s_values)
combinations_t = itertools.product(t_values)
combinations_s_a = itertools.product(s_values, a_values)
combinations_f_r_w = itertools.product(f_values, r_values, w_values)
combinations_w = itertools.product(w_values)
combinations_r = itertools.product(r_values)

#### Inputdata

In [5]:
# Hardkodet inn fuelpris per MJ per fueltype 1-6
fuel_prices = {1: 0.171, 2:0.163, 3:0.514, 4:0.328, 5:0.222, 6:0.297}




# fuel_dict = {
#     1: (0.178, 0.094, 'fossil'),      # MGO
#     2: (0.146, 0.073, 'fossil'),      # LNG
#     3: (0.491, 0.261, 'green_fuels'), # GREEN METHANOL
#     4: (0.324, 0.178, 'green_fuels'), # GREEN NH3
#     5: (0.278, 0.147, 'bio'),         # B30
#     6: (0.245, 0.130, 'bio'),         # B20
#     7: (0.228, 0.121, 'bio'),         # B15
#     8: (0.201, 0.106, 'bio'),         # B7
#     9: (0.301, 0.129, 'green_fuels')  # Helelektrisk
# }
 # Example fuel bounds with group information


# Karbonutslipp per MJ som er brukt for hvert system
emissions_carbon_tonn_per_MJ_wtw = {1: 0.000182, 2: 0.000153, 3: 0.000033, 4: 0.000033, 5: 0.000145, 6: 0.000122, 7: 0.000026, 8: 0.000026, 9: 0.000033, 10: 0.000033, 11: 0.000104, 12: 0.000053, 13:0.000058 }
emissions_carbon_tonn_per_MJ_lind = {1: 0.000182, 2: 0.000153, 3: 0.000002, 4: 0.000011, 5: 0.000145, 6: 0.000122, 7: 0.000002, 8: 0.000008, 9: 0.000002, 10: 0.000011, 11: 0.000102, 12: 0.000053, 13:0.000058 }
emissions_carbon_tonn_per_MJ = emissions_carbon_tonn_per_MJ_wtw


# Regne ut reduksjonskrav for karbonutslipp
init_ems_tonn = 137000 # 137000

reductions_10_kutt = {0: 1, 1: 0.98, 2: 0.96, 3: 0.94, 4: 0.92, 5: 0.90 } 
reductions_20_kutt = {0: 1, 1: 0.96, 2: 0.92, 3: 0.88, 4: 0.84, 5: 0.80} 
reductions_30_kutt = {0: 1, 1: 0.94, 2: 0.88, 3: 0.82, 4: 0.76, 5: 0.70} 
reductions_40_kutt = {0: 1, 1: 0.92, 2: 0.84, 3: 0.76, 4: 0.68, 5: 0.60} 
reductions_50_kutt = {0: 1, 1: 0.90, 2: 0.80, 3: 0.70, 4: 0.60, 5: 0.50} 
reductions_60_kutt = {0: 1, 1: 0.88, 2: 0.76, 3: 0.64, 4: 0.52, 5: 0.40} 
reductions_70_kutt = {0: 1, 1: 0.86, 2: 0.72, 3: 0.58, 4: 0.44, 5: 0.30} 
reductions_80_kutt = {0: 1, 1: 0.84, 2: 0.68, 3: 0.52, 4: 0.36, 5: 0.20} 
reductions_0_kutt = {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1}
reductions_imo_kutt = {0: 1, 1: 0.80, 2: 0.80, 3: 0.30, 4: 0.30, 5: 0.20}
reductions_lindstad_kutt = {0: 1, 1: 0.80, 2: 0.80, 3: 0.30, 4: 0.30, 5: 0.05}
# Velg hvilket scenario som skal kjøres
reduction_requirements = reductions_0_kutt

capacity_systems = {1: 100, 2: 100, 3: 100, 4: 100, 5: 70, 6: 70, 7: 70, 8: 70, 9: 90, 10: 90, 11: 70, 12: 90, 13: 90}

#### Lese inn data fra rutegenereringsprogrammet

In [6]:


# # Leser inn nødvendig MJ på å kjøre hver rute
mj_route = pd.read_csv(f'../route_generation_{route_case}/{mappenavn}{startfil}mj_route.csv')

# # Leser inn tid på å kjøre hver rute
hours_route = pd.read_csv(f'../route_generation_{route_case}/{mappenavn}{startfil}hours_route.csv')
#hours_route_el = pd.read_csv('../route_generation/generated_datafiles_reduced/hours_route_strøm.csv')

#demand_el = pd.read_csv('../route_generation/generated_datafiles_reduced/demand_strøm.csv')
demand = pd.read_csv(f'../route_generation_{route_case}/{mappenavn}{startfil}demand.csv')

routes_file = f"../route_generation_{route_case}/{mappenavn}{startfil}routes.csv"
excel_file = '../run_model/Parameterdata.xlsx'
platform_demand = f"../route_generation_{route_case}/{mappenavn}output_platforms_demand.csv"
csv_platform_visits = f'../route_generation_{route_case}/{mappenavn}output_platforms_visits.csv'

# Skriver verdiene tilbake i excelfilen for alle parametrene
output_excel_path = '../run_model/Parameterdata.xlsx'



##### Generere P_w

In [7]:
df_probability = pd.DataFrame(combinations_w, columns=['O'])

df_probability['Value'] = 1/len(list(w_values))

df_probability.head()


,O,Value
0,1,0.037037
1,2,0.037037
2,3,0.037037
3,4,0.037037
4,5,0.037037


In [8]:

with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_probability.to_excel(writer, sheet_name='Probability', index=False)

In [9]:
distance_dict = {}

for idx, value in enumerate(mj_route,1):
    distance_dict[idx] = value

print(distance_dict)

{1: 'mj_route'}


##### Distance dict for compatibility og cost of fuel

In [10]:
# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
distance_dict = mj_route['mj_route'].to_dict()
print(distance_dict)
# lager egne distanser for elektrisk

# specific_distance_dict = mj_route_el['mj_route'].to_dict()
#specific_distance_dict= specific_distance_dict['mj_route']
# electric_distance_dict = {key + 1: value for key, value in specific_distance_dict.items()}

#print(distance_dict)
distance_dict = {key + 1: value for key, value in enumerate(distance_dict.values())}
print(len(distance_dict))
print(distance_dict)


{0: 269610.4, 1: 285944.8, 2: 242590.0, 3: 320381.2, 4: 311261.2, 5: 258570.4, 6: 257464.8, 7: 288541.2, 8: 311156.4, 9: 321932.4, 10: 337304.8, 11: 349875.2, 12: 293160.4, 13: 348551.6, 14: 328231.6, 15: 295289.2, 16: 272995.2, 17: 315191.6, 18: 351886.8, 19: 356822.8, 20: 395155.2, 21: 310374.8, 22: 344886.0, 23: 354566.0, 24: 323155.2, 25: 341489.6, 26: 386006.0, 27: 324621.2, 28: 337637.2, 29: 350289.6, 30: 327462.8, 31: 319382.8, 32: 267400.4, 33: 283494.8, 34: 329062.8, 35: 322158.0, 36: 330922.4, 37: 346454.8, 38: 339242.4, 39: 331911.6, 40: 341846.0, 41: 379722.4, 42: 333789.2, 43: 334313.6, 44: 347286.0, 45: 320711.6, 46: 322006.0, 47: 356362.4, 48: 347229.2, 49: 349033.6, 50: 361606.0, 51: 286275.2, 52: 331591.6, 53: 329886.8, 54: 337222.8, 55: 352595.2, 56: 306246.0, 57: 344541.2, 58: 349797.2, 59: 364289.6, 60: 385549.2, 61: 389033.6, 62: 426406.0, 63: 325168.8, 64: 338941.2, 65: 339077.2, 66: 398385.2, 67: 432816.4, 68: 442576.4, 69: 411154.0, 70: 429408.4, 71: 473936.4, 7

##### Compatibility RS

In [11]:
# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
demand_dict = demand['Demand'].to_dict()
#demand_dict = demand_dict['Demand']

# lager egne distanser for elektrisk

#specific_demand_dict = demand_el['Demand'].to_dict()
#specific_demand_dict= specific_demand_dict['Demand']

#electric_demand_dict = {key + 1: value for key, value in specific_demand_dict.items()}

demand_dict = {key + 1: value for key, value in demand_dict.items()}




In [12]:
# Lage combinations for rs
# For hver rad med r s, sjekke om kapasiteten i s er større enn eller lik demanden til r. Sett til 1 hvis true, 0 ellers

# Create a DataFrame from the combinations
df_routecap = pd.DataFrame(combinations_r_s, columns=['R', 'S'])

# Display the first few rows of the DataFrame
print(df_routecap.head())

df_routecap['Value'] = 0

# Iterate over the DataFrame
for index, row in df_routecap.iterrows():
    r = row['R']
    s = row['S']
    if demand_dict.get(r, 0) <= capacity_systems.get(s, float('inf')):
        df_routecap.at[index, 'Value'] = 1
    # if s == 11 and electric_demand_dict.get(r, 0) <= capacity_systems.get(s, float('inf')):
    #     df_routecap.at[index, 'Value'] = 1

#print(df_routecap)

   R  S
0  1  1
1  1  2
2  1  3
3  1  4
4  1  5


In [13]:
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_routecap.to_excel(writer, sheet_name='Compatibility rs', index=False)


### Compatibility fs oppdateres manuelt!

#### Demand for set covering

In [14]:
df_demand = pd.DataFrame(list(demand_dict.items()), columns=['R', 'Value'])

print(df_demand)

       R  Value
0      1   40.0
1      2   14.0
2      3   29.0
3      4   18.0
4      5   18.0
..   ...    ...
853  854   89.0
854  855   93.0
855  856   89.0
856  857   89.0
857  858   93.0

[858 rows x 2 columns]


In [15]:
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_demand.to_excel(writer, sheet_name='Demand route', index=False)

##### Cost of fuel per route

In [16]:

# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
distance_dict = mj_route['mj_route'].to_dict()
#distance_dict = distance_dict['mj_route']

# lager egne distanser for elektrisk

# specific_distance_dict = mj_route_el['mj_route'].to_dict()
# #specific_distance_dict= specific_distance_dict['mj_route']
# electric_distance_dict = {key + 1: value for key, value in specific_distance_dict.items()}

#print(distance_dict)
distance_dict = {key + 1: value for key, value in distance_dict.items()}
print(len(distance_dict))

print(distance_dict)
#print(len(electric_distance_dict))

858
{1: 269610.4, 2: 285944.8, 3: 242590.0, 4: 320381.2, 5: 311261.2, 6: 258570.4, 7: 257464.8, 8: 288541.2, 9: 311156.4, 10: 321932.4, 11: 337304.8, 12: 349875.2, 13: 293160.4, 14: 348551.6, 15: 328231.6, 16: 295289.2, 17: 272995.2, 18: 315191.6, 19: 351886.8, 20: 356822.8, 21: 395155.2, 22: 310374.8, 23: 344886.0, 24: 354566.0, 25: 323155.2, 26: 341489.6, 27: 386006.0, 28: 324621.2, 29: 337637.2, 30: 350289.6, 31: 327462.8, 32: 319382.8, 33: 267400.4, 34: 283494.8, 35: 329062.8, 36: 322158.0, 37: 330922.4, 38: 346454.8, 39: 339242.4, 40: 331911.6, 41: 341846.0, 42: 379722.4, 43: 333789.2, 44: 334313.6, 45: 347286.0, 46: 320711.6, 47: 322006.0, 48: 356362.4, 49: 347229.2, 50: 349033.6, 51: 361606.0, 52: 286275.2, 53: 331591.6, 54: 329886.8, 55: 337222.8, 56: 352595.2, 57: 306246.0, 58: 344541.2, 59: 349797.2, 60: 364289.6, 61: 385549.2, 62: 389033.6, 63: 426406.0, 64: 325168.8, 65: 338941.2, 66: 339077.2, 67: 398385.2, 68: 432816.4, 69: 442576.4, 70: 411154.0, 71: 429408.4, 72: 473936

In [17]:
# Convert dictionaries to DataFrames
df_distance = pd.DataFrame(distance_dict.items(), columns=['R', 'Value'])

# Sort DataFrames by 'R' values
df_distance = df_distance.sort_values(by='R')

with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_distance.to_excel(writer, sheet_name='Distance', index=False)

Generere alle kombinasjoner av R og F

In [18]:
# Create a DataFrame from the combinations
df_fuelprice = pd.DataFrame(combinations_f_r, columns=['F', 'R'])

# Display the first few rows of the DataFrame
print(df_fuelprice.head())

   F  R
0  1  1
1  1  2
2  1  3
3  1  4
4  1  5


#### Fuelcost

In [19]:


def calculate_value(row):
    distance = distance_dict[row['R']]
    return fuel_prices[row['F']] * distance

# Legger til verdiene i Value-kolonne i dataframen
df_fuelprice['Value'] = df_fuelprice.apply(calculate_value, axis=1)

# df_fuelprice

##### Fuelcost 2

In [20]:
fuel_price2_df = pd.read_csv(fuel_price_csv)

fuel_price2_df.head()

,F,T,S,Value
0,1,2,1,0.1730
1,1,3,1,0.1746
2,1,4,1,0.1761
3,1,5,1,0.1777
4,2,2,1,0.1586


##### Skriver Cost of fuel til excel

In [21]:
# Using ExcelWriter to write to the existing Excel file
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    fuel_price2_df.to_excel(writer, sheet_name='Cost of fuel 2', index=False)

# Using ExcelWriter to write to the existing Excel file
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_fuelprice.to_excel(writer, sheet_name='Cost of fuel 1', index=False)

##### EMISSIONS FOR FUEL TYPE

Generere alle kombinasjoner av F og R i kolonnene


In [22]:
# Create a DataFrame from the combinations
df_ems = pd.DataFrame(combinations_ems, columns=['S', 'R'])

# Display the first few rows of the DataFrame
print(df_ems.head())

   S  R
0  1  1
1  1  2
2  1  3
3  1  4
4  1  5


In [23]:

def calculate_emissions(row):
    distance =  distance_dict[row['R']]
    return emissions_carbon_tonn_per_MJ[row['S']] * distance



# Apply the function to create a new column 'Value'
df_ems['Value'] = df_ems.apply(calculate_emissions, axis=1)


# Skriver dataframen med emissions til parameter_data excel filen
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_ems.to_excel(writer, sheet_name='Emissions', index=False)


##### MAX EMISSIONS PER TIME PERIOD

In [24]:
# Create a DataFrame from the combinations
df_max_ems = pd.DataFrame(combinations_t, columns=['T'])

In [25]:


# Regner ut max emissions gitt uslippsreduksjonsscenario. Endre scenario-funksjon etter hvilket scenario vi vil kjøre med
def calculate_max_emissions(row):
    return (reduction_requirements[row['T']]) * init_ems_tonn

# Legger til verdiene i Value-kolonne i dataframen
df_max_ems['Value'] = df_max_ems.apply(calculate_max_emissions, axis=1)

# Skriver dataframen med emissions til parameter_data excel filen
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_max_ems.to_excel(writer, sheet_name='Max Emissions', index=False)

df_max_ems

,T,Value
0,0,137000
1,1,137000
2,2,137000
3,3,137000
4,4,137000
5,5,137000


##### TIME USED

In [26]:
# Create T_rs
combinations_r_s = itertools.product(r_values, s_values)
df_timeused = pd.DataFrame(combinations_r_s, columns=['R', 'S'])

# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
route_dict = hours_route['Hours'].to_dict()
hours_route_dict = {key + 1: value for key, value in route_dict.items()}
print(hours_route_dict)


def calculate_timeused(row):
    value =  hours_route_dict[row['R']]
    return value


df_timeused['Value'] = df_timeused.apply(calculate_timeused, axis=1)

print(df_timeused.head())
# Skriver dataframe med emissions til parameter_data excel filen
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_timeused.to_excel(writer, sheet_name='Time used', index=False)


{1: 48.0, 2: 48.0, 3: 48.0, 4: 48.0, 5: 48.0, 6: 48.0, 7: 48.0, 8: 48.0, 9: 48.0, 10: 48.0, 11: 48.0, 12: 48.0, 13: 48.0, 14: 48.0, 15: 48.0, 16: 48.0, 17: 48.0, 18: 48.0, 19: 48.0, 20: 48.0, 21: 72.0, 22: 48.0, 23: 48.0, 24: 48.0, 25: 48.0, 26: 48.0, 27: 48.0, 28: 48.0, 29: 48.0, 30: 48.0, 31: 48.0, 32: 48.0, 33: 48.0, 34: 48.0, 35: 48.0, 36: 48.0, 37: 48.0, 38: 48.0, 39: 48.0, 40: 48.0, 41: 48.0, 42: 48.0, 43: 48.0, 44: 48.0, 45: 48.0, 46: 48.0, 47: 48.0, 48: 48.0, 49: 48.0, 50: 48.0, 51: 48.0, 52: 48.0, 53: 48.0, 54: 48.0, 55: 48.0, 56: 48.0, 57: 48.0, 58: 48.0, 59: 48.0, 60: 48.0, 61: 48.0, 62: 48.0, 63: 72.0, 64: 48.0, 65: 48.0, 66: 48.0, 67: 72.0, 68: 72.0, 69: 72.0, 70: 72.0, 71: 72.0, 72: 72.0, 73: 72.0, 74: 72.0, 75: 72.0, 76: 72.0, 77: 72.0, 78: 72.0, 79: 72.0, 80: 72.0, 81: 72.0, 82: 72.0, 83: 72.0, 84: 72.0, 85: 72.0, 86: 72.0, 87: 72.0, 88: 72.0, 89: 72.0, 90: 72.0, 91: 72.0, 92: 72.0, 93: 72.0, 94: 72.0, 95: 72.0, 96: 72.0, 97: 72.0, 98: 72.0, 99: 72.0, 100: 72.0, 101: 48

#### Installations in route

In [27]:
import csv


# Open the existing Excel workbook
workbook = openpyxl.load_workbook(output_excel_path)
sheet = workbook["Installations in route"]

# Open the CSV file in text mode and read all lines
with open(routes_file, 'r') as file:
    lines = file.readlines()

# Add column headers if necessary
if not sheet['A1'].value:
    sheet['A1'] = "i"
    sheet['B1'] = "r"
    sheet['C1'] = "Value"

# Initialize the row index for Excel
row_index = 2

# Initialize an empty dictionary to store the platform mapping
platform_mapping = {}

# Open the CSV file and read the contents
with open(platform_demand, newline='') as file:
    reader = csv.reader(file, delimiter=';')
    next(reader)  # Skip the header row
    for row in reader:
        platform = row[0]  # Platform name is in the first column
        # Assign a number to each platform
        platform_mapping[platform] = len(platform_mapping) + 1

print(platform_mapping)


{'DAB': 1, 'DRA': 2, 'DSA': 3, 'GKR': 4, 'GUD': 5, 'JSF': 6, 'GRA': 7, 'HDA': 8, 'SLA': 9, 'SLB': 10, 'TEB': 11}


In [28]:

platform_mapping_reverse = {v: k for k, v in platform_mapping.items()}

length = len(platform_mapping) +1

# Process each line in the CSV file
for idx, line in enumerate(lines, start=1):
    for i in range(1, length):
        sheet.cell(row=row_index, column=2, value=idx)
        sheet.cell(row=row_index, column=1, value=i)
        platform_name = platform_mapping_reverse.get(i, "")  # Get the platform name
        if platform_name in line:
            sheet.cell(row=row_index, column=3, value=1)
        else:
            sheet.cell(row=row_index, column=3, value=0)
        row_index += 1

workbook.save(excel_file)
print("Data has been successfully written.")

Data has been successfully written.


In [29]:

visits_per_week = {}
with open(csv_platform_visits, newline='') as file:
    reader = csv.reader(file, delimiter=';')
    next(reader)  # Skip the header row
    i = 1
    for row in reader:
        platform = row[0]  # Platform name is in the first column
        # Assign a number to each platform
        # Assign a number to each platform
        visits_per_week[i] = row[1]
        i += 1

i_values = range(1, len(platform_mapping)+1)
print(visits_per_week)

combinations_i_t = itertools.product(i_values, t_values)
df_demand = pd.DataFrame(combinations_i_t, columns=['I', 'T'])
# Iterate over rows in the DataFrame
df_demand['Value'] = df_demand['I'].map(lambda x: visits_per_week[x])



df_demand

{1: '4', 2: '1', 3: '3', 4: '1', 5: '1', 6: '4', 7: '3', 8: '4', 9: '1', 10: '2', 11: '3'}


,I,T,Value
0,1,0,4
1,1,1,4
2,1,2,4
3,1,3,4
4,1,4,4
...,...,...,...
61,11,1,3
62,11,2,3
63,11,3,3
64,11,4,3


In [30]:
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_demand.to_excel(writer, sheet_name='Demand', index=False)